<a href="https://colab.research.google.com/github/srirambandi/ai/blob/master/examples/mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# when running in colab notebooks, first install library
!pip install import-ai
# upload respective dataset manually from examples directory of the library or download as below
!apt install subversion
!svn checkout https://github.com/srirambandi/ai/trunk/examples/MNIST

In [2]:
import ai
import numpy as np

In [3]:
def load_data(file):
    dict = np.load(file, allow_pickle=True)
    return dict

In [4]:
train_file = 'MNIST/train.npy'
test_file = 'MNIST/test.npy'

In [5]:
ai.manual_seed(2357)

In [6]:
class MNIST(ai.Module):
    def __init__(self):
        self.conv1 = ai.Conv2d(1, 8, kernel_size=3, stride=1)
        self.conv2 = ai.Conv2d(8, 16, kernel_size=3, stride=1)
        self.pool = ai.Maxpool2d(kernel_size=2, stride=2)
        self.drop1 = ai.Dropout(p=0.75)
        self.fc1 = ai.Linear(2304, 128)
        self.drop2 = ai.Dropout(p=0.5)
        self.fc2 = ai.Linear(128, 10)
        
    def forward(self, x):
        o1 = ai.G.relu(self.conv1(x))
        o2 = ai.G.relu(self.conv2(o1))
        o3 = self.drop1(self.pool(o2))
        o4 = self.drop2(ai.G.relu(self.fc1(o3)))
        o5 = ai.G.softmax(self.fc2(o4))
        
        return o5

In [7]:
mnist = MNIST()
print(mnist)

MNIST(
  conv1: Conv2d(1, 8, kernel_size=(3, 3), stride=(1, 1), padding=(0, 0), bias=True)
  conv2: Conv2d(8, 16, kernel_size=(3, 3), stride=(1, 1), padding=(0, 0), bias=True)
  pool: Maxpool2d(kernel_size=(2, 2), stride=(2, 2), padding=(0, 0))
  drop1: Dropout(p=0.75)
  fc1: Linear(input_features=2304, output_features=128, bias=True)
  drop2: Dropout(p=0.5)
  fc2: Linear(input_features=128, output_features=10, bias=True)
)


In [8]:
L = ai.Loss(loss_fn='CrossEntropyLoss')
optim = ai.Optimizer(mnist.parameters(), optim_fn='Adadelta', lr=1e-3)

In [ ]:
train_dict = load_data(train_file)
inputs = train_dict.item()['data']
outputs = train_dict.item()['labels']
del train_dict

In [10]:
it, epoch = 0, 0
m = 128

In [11]:
def evaluate():
    ai.G.grad_mode = False
    file = test_file
    dict = load_data(file)
    inputs = dict.item()['data']
    outputs = dict.item()['labels']
    correct, total = 0, 0
    test_m = m
    for batch in range(int(len(outputs) / m)):
        input = inputs[batch * test_m : (batch + 1) * test_m].reshape(test_m, 1, 28, 28) / 255
        input =  np.stack([_ for _ in input], axis = -1)
        output = np.array(outputs[batch * test_m : (batch + 1) * test_m])
        scores = mnist.forward(input)
        preds = np.argmax(scores.data, axis=0)
        correct += np.sum(np.equal(output, preds))
        total += test_m
    accuracy = float(correct / total)
    ai.G.grad_mode = True
    return accuracy

In [12]:
while epoch < 15:
    epoch += 1
    it = 0

    for batch in range(int(len(outputs) / m)):
    
        input = inputs[batch * m : (batch + 1) * m].reshape(m, 1, 28, 28) / 255.
        input =  np.stack([_ for _ in input], axis = -1)
        output = outputs[batch * m : (batch + 1) * m]
        onehot = np.zeros((10, m))
        for _ in range(m):
            onehot[output[_], _] = 1.0

        scores = mnist.forward(input)
        loss = L.loss(scores, onehot)
        loss.backward()

        optim.step()        # update parameters with optimization functions
        optim.zero_grad()   # clearing the backprop list and resetting the gradients to zero

        if it%50 == 0:
            print('epoch: {}, iter: {}, loss: {}'.format(epoch, it, loss.data[0, 0]))
        it += 1

    print('Epoch {} completed. Accuracy: {:.2%} \n'.format(epoch, evaluate()))

using Adadelta
epoch: 1, iter: 0, loss: 2.3012260680220527
epoch: 1, iter: 50, loss: 0.6679869887537331
epoch: 1, iter: 100, loss: 0.4401506684320097
epoch: 1, iter: 150, loss: 0.4883632865960655
epoch: 1, iter: 200, loss: 0.25288718433194224
epoch: 1, iter: 250, loss: 0.3267003642500389
epoch: 1, iter: 300, loss: 0.25623870250947844
epoch: 1, iter: 350, loss: 0.33838382493391467
epoch: 1, iter: 400, loss: 0.21898741959533843
epoch: 1, iter: 450, loss: 0.2885756179330047
Epoch 1 completed. Accuracy: 95.90% 

epoch: 2, iter: 0, loss: 0.14540136723455022
epoch: 2, iter: 50, loss: 0.2179695241457375
epoch: 2, iter: 100, loss: 0.12919534264492227
epoch: 2, iter: 150, loss: 0.17735789293460685
epoch: 2, iter: 200, loss: 0.165407585434569
epoch: 2, iter: 250, loss: 0.24592577568023366
epoch: 2, iter: 300, loss: 0.16670144925039734
epoch: 2, iter: 350, loss: 0.306696637449704
epoch: 2, iter: 400, loss: 0.17189841723073077
epoch: 2, iter: 450, loss: 0.22262930797638847
Epoch 2 completed. Accur

In [13]:
mnist.save()

saving model...
Successfully saved model in MNIST.npy
